# Prompt selection and testing

This notebook chooses the most appropriate prompt and prompt structure for the OCR correction. 

In [21]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, compare_request_configurations, generate_model_configs
from evaluate import load
from evaluation_funcs import evaluate_correction_performance, evaluate_correction_performance_folders, get_metric_error_reduction
import seaborn as sns
import matplotlib.pyplot as plt
from helper_functions import files_to_df_func, evaluate_ner, calculate_entity_similarity, repeat_prompt_experiment
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re


dev_data_folder = 'data/dev_data'
dev_transcripts = os.path.join(dev_data_folder, 'dev_data_transcript')
dev_raw_ocr_folder =  os.path.join(dev_data_folder,'dev_raw_ocr' )
dev_system_message_folder = os.path.join(dev_data_folder,'dev_system_message_variants' )

#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv(os.path.join(dev_data_folder,'dev_data_raw.csv'))


#for saving data to be used in the analysis
if not os.path.exists('data/analysis'):
    os.makedirs('data/analysis')

## Evaluate system prompt tests

We evaluate the system prompts below to see if thre is any significant difference between the prompts


In [7]:
wer = load("wer")
cer = load("cer")

# Evaluating models on the test set

Having identified two different prompts and that the prompts appear to work better when places after the text we can now compare the different models


The below code creates the basic configuration dictionaries for each model and then fills in the with the two different prompt messages creating a single list of all basic prompt/model configurations. It then calls all the LLM's and saves the results.
This works in series so takes a while.

## Create API configurations

In [28]:
#Create the prompt/system message using the best performing from the previous section

full_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. The text is from an english newspaper in the 1800's. The text may be an advert or article and may be missing the beggining or end. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

instruct_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

boros_basic  = "{content_html}"+f""" \n \n """ +"Correct the text"

boros_complex  ="{content_html}"+f""" \n \n """ + f"""Please assist with reviewing and correcting errors in texts produced by automatic transcription (OCR) of historical documents.
Your task is to carefully examine the following text and correct any mistakes introduced by the OCR software. 
Do not write anything else than the corrected text."""

In [47]:
groq_alt_endpoint = {'alt_endpoint':{'base_url':'https://api.groq.com/openai/v1',
                     'api_key':os.getenv("GROQ_API_KEY")}}

basic_model_configs = pd.DataFrame({
    'get_response_func': [get_response_openai, get_response_openai, get_response_anthropic, get_response_anthropic, 
                          get_response_openai, get_response_openai, get_response_openai], 
    'engine': ['gpt-3.5-turbo', 'gpt-4-turbo-preview', "claude-3-haiku-20240307", "claude-3-opus-20240229", 
               'mixtral-8x7b-32768', 'llama2-70b-4096', 'gemma-7b-it'],
    'rate_limit':[160e3, 80e3, 100e3, 40e3, 18e3, 30e3, 30e3],
    'additional_args': [
        {}, {}, {}, {}, 
        groq_alt_endpoint, 
        groq_alt_endpoint, 
        groq_alt_endpoint
    ]
})

full_model_configs = generate_model_configs(basic_model_configs, full_prompt, 'full')
instruct_model_configs = generate_model_configs(basic_model_configs, instruct_prompt, 'instruct')

#I think on reflection I only need to compare boros complex on gpt-4 as this was the best performer in their paper
boros_configs = [
    (get_response_openai, 'gpt-4-turbo-preview', boros_complex, "boros_complex_"),
   # (get_response_openai, 'gpt-4-turbo-preview', boros_basic, "boros_basic_"),
  #  (get_response_anthropic, "claude-3-opus-20240229", boros_complex, "boros_complex_")
]

boros_list = [
    create_config_dict_func(
        get_response_func=config[0],
        rate_limiter=RateLimiter(80e3),
        engine=config[1],
        system_message_template="",
        prompt_template=config[2],
        additional_args={"response_name": config[3]}
    )
    for config in boros_configs
]

model_configs = full_model_configs + instruct_model_configs + boros_list



## Perform all API calls

The below section is what actually calls the API, the code points to the folders where the raw OCR is and provides a path to where the corrected text should be stored.

### NCSE

In [40]:
corrected_folder = 'data/transcription_returned_ocr/corrected_folder'
#
# This naming business needs to be cleaned up so the actual article ID is used. until then just have the following mess
#
# 

test_data_new = pd.read_csv(os.path.join(dev_data_folder,'transcription_raw_ocr.csv'))
test_data_new = test_data_new.loc[test_data_new ['file_name'].isin(files_to_df_func(dev_raw_ocr_folder )['file_name'])]

#used on the original devset
#compare_request_configurations(dev_data_df, model_configs, folder_path='./data/dev_corrected_base')
#This goes through the list of articles that have been transcribed, checks to see if there is a corrected version and if not generates it
compare_request_configurations(test_data_new, model_configs, folder_path=corrected_folder)

### Sydney Morning Herald

### Chronicalling America

## Evaluate the prompts across all models

On the smaller models, Full is worse than instruct on the larger models the reverse. Maybe this is related to ability to 'focus' or hold isntructions in memory?

In [32]:
corrected_folder = dev_system_message_folder 

performance_eval =  evaluate_correction_performance_folders(corrected_folder, dev_transcripts, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt') &
                     (performance_eval['type']!='gpt3_boros_blank_gpt-3.5-turbo'),:]

In [34]:
error_reduction = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)


error_reduction.to_csv('data/analysis/error_reduction.csv', index=False)


error_reduction.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER  \
                                                     mean    50%    mean   
type                                                                       
expert_recover_text_prompt_claude-3-haiku-20240307  -3.36  13.64 -296.74   
nosm_expert_recover_publication_text_prompt_cla...   4.55  18.18 -294.91   
expert_recover_publication_text_prompt_claude-3...  17.73  35.45 -209.70   
nosm_expert_recover_text_prompt_claude-3-haiku-...   1.70  11.65 -297.54   
expert_recover_prompt_claude-3-haiku-20240307       39.84  48.11  -90.64   
...                                                   ...    ...     ...   
nosm_full_context_prompt_gpt-4-turbo-preview        73.21  77.19   54.38   
expert_recover_instructions_prompt_claude-3-opu...  73.67  80.46   46.82   
nosm_expert_basic_prompt_gpt-4-turbo-preview        73.61  77.59   55.55   
nosm_expert_recover_prompt_gpt-4-turbo-preview      72.03  75.86   49.27   
nosm_full_context_prompt_claude-3-opus-20240229     73.42  80.46   58.95   

                                                           lev_dist          
                                                       50%     mean     50%  
type                                                                         
expert_recover_text_prompt_claude-3-haiku-20240307 -197.56  -282.00 -177.78  
nosm_expert_recover_publication_text_prompt_cla... -152.94  -279.84 -144.31  
expert_recover_publication_text_prompt_claude-3... -115.22  -198.48 -110.20  
nosm_expert_recover_text_prompt_claude-3-haiku-... -107.24  -280.89 -102.16  
expert_recover_prompt_claude-3-haiku-20240307       -39.82   -85.44  -36.21  
...                                                    ...      ...     ...  
nosm_full_context_prompt_gpt-4-turbo-preview         63.78    52.09   62.23  
expert_recover_instructions_prompt_claude-3-opu...   68.01    45.19   62.87  
nosm_expert_basic_prompt_gpt-4-turbo-preview         62.80    53.41   63.64  
nosm_expert_recover_prompt_gpt-4-turbo-preview       65.55    47.03   63.79  
nosm_full_context_prompt_claude-3-opus-20240229      68.78    57.84   66.36  

[64 rows x 6 columns]

In [41]:
performance_eval2 = performance_eval.copy()
performance_eval2['type'] = performance_eval2['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")
performance_eval2['model'] = performance_eval2['type'].str.split('_').str[-1]

#The below line allows you to look at an individual model
#performance_eval2 = performance_eval2.loc[performance_eval2['model'].str.contains('gpt-4')]

performance_eval2.drop(columns = 'File Name').groupby(['type', 'model']).describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.23   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.23   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.24   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.24   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.25   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.57   
expert_recover_publication_text_prompt_haiku       haiku                   0.71   
nosm_expert_recover_text_prompt_haiku              haiku                   0.81   
expert_recover_text_prompt_haiku                   haiku                   0.83   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.81   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.15   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.12   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.16   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.11   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.51   
expert_recover_publication_text_prompt_haiku       haiku                   0.51   
nosm_expert_recover_text_prompt_haiku              haiku                   0.73   
expert_recover_text_prompt_haiku                   haiku                   0.90   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.68   

                                                                            CER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.13   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.14   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.14   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.15   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.44   
expert_recover_publication_text_prompt_haiku       haiku                   0.58   
nosm_expert_recover_text_prompt_haiku              haiku                   0.65   
expert_recover_text_prompt_haiku                   haiku                   0.68   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.66   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.06   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.06   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.05   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.07   
nosm_f

# Test on the test set
confusing but you know what I mean, This is jsut to make sure it all works as expected

In [19]:
transcribed_files = 'data/transcription_returned_ocr/transcription_files'

corrected_folder = 'data/transcription_returned_ocr/corrected_folder'

raw_folder = "/home/jonno/redigitalize/data/transcription_raw_ocr"

performance_eval =  evaluate_correction_performance_folders(corrected_folder, transcribed_files, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt') &
                     (performance_eval['type']!='gpt3_boros_blank_gpt-3.5-turbo'),:]

In [20]:
performance_eval.describe()

,WER,CER,lev_dist
count,324.000000,324.000000,324.000000
mean,0.427249,0.320615,674.836420
std,1.404056,1.181777,1563.578346
min,0.000000,0.000000,1.000000
25%,0.047231,0.017892,55.750000
50%,0.160026,0.080230,130.000000
75%,0.383333,0.224831,446.000000
max,17.497006,13.418564,15369.000000


In [21]:
raw_dev_ocr_scores = evaluate_correction_performance(raw_folder, transcribed_files, wer, cer, 'raw_ocr')

error_reduction_df = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)

error_reduction_df.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER             CER        lev_dist  \
                                         mean    50%     mean    50%     mean   
type                                                                            
full__gemma-7b-it                       10.12  27.19  -253.41 -60.56  -249.00   
instruct__gemma-7b-it                   24.95  39.52  -172.24 -37.54  -168.29   
full__mixtral-8x7b-32768                36.60  43.54  -133.47 -37.87  -134.53   
instruct__mixtral-8x7b-32768            44.84  55.42   -81.32 -27.66   -79.69   
full__claude-3-haiku-20240307           21.36  57.17  -295.09  -4.68  -285.72   
instruct__llama2-70b-4096              -15.26  42.98  -374.12  -0.88  -356.15   
instruct__claude-3-haiku-20240307       40.83  57.99  -189.58   9.43  -183.53   
full__llama2-70b-4096                 -161.34  58.97  -991.54  15.29  -909.02   
full__gpt-3.5-turbo                     59.10  70.00   -45.12  39.74   -42.64   
instruct__gpt-3.5-turbo                 56.68  75.36   -17.02  48.57   -14.90   
boros_complex__claude-3-opus-20240229   25.70  81.12  -192.53  58.70  -173.41   
full__gpt-4-turbo-preview               77.58  80.38    59.46  62.84    57.26   
boros_basic__gpt-4-turbo-preview        76.37  77.21    55.96  62.65    54.33   
instruct__gpt-4-turbo-preview           78.17  79.38    61.62  66.50    59.28   
boros_complex__gpt-4-turbo-preview      79.80  84.22    66.96  70.37    64.79   
full__claude-3-opus-20240229            -4.45  82.24  -375.41  78.35  -335.13   
claude_temp_claude-3-opus-20240229      22.89  82.24  -231.82  78.64  -205.53   
instruct__claude-3-opus-20240229      -136.72  79.25 -1519.07  78.89 -1449.43   

                                              
                                         50%  
type                                          
full__gemma-7b-it                     -53.34  
instruct__gemma-7b-it                 -35.61  
full__mixtral-8x7b-32768              -33.88  
instruct__mixtral-8x7b-32768          -28.71  
full__claude-3-haiku-20240307          -2.99  
instruct__llama2-70b-4096              -0.36  
instruct__claude-3-haiku-20240307       9.42  
full__llama2-70b-4096                  14.65  
full__gpt-3.5-turbo                    39.18  
instruct__gpt-3.5-turbo                47.85  
boros_complex__claude-3-opus-20240229  58.29  
full__gpt-4-turbo-preview              61.85  
boros_basic__gpt-4-turbo-preview       62.33  
instruct__gpt-4-turbo-preview          63.97  
boros_complex__gpt-4-turbo-preview     67.48  
full__claude-3-opus-20240229           75.89  
claude_temp_claude-3-opus-20240229     76.17  
instruct__claude-3-opus-20240229       76.44